In [1]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.models import Model
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold,CategoricalStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
import sys
from sklearn import linear_model
import matplotlib.pyplot as plt
print('The scikit-learn version is {}.'.format(sklearn.__version__))

Using Theano backend.
Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


The scikit-learn version is 0.19.dev1.1.


In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from ccfd_dnn.utils import get_engine

In [2]:
module_path = os.path.abspath(os.path.join('..'))
print module_path
if module_path not in sys.path:
    sys.path.append(module_path)
    

/home/botty/Documents/CCFD


In [14]:
table = "data_fraud"
total_samples = pd.read_sql_query("select count(*) from {table}".format(table=table),engine).values

In [17]:
total_samples[0][0]

4732132

In [7]:
table = "data_fraud_little"
pd.read_sql_query("select count(*) from {table}".format(table=table),engine)

,count
0,600000


In [4]:
# load dataset
table = "data_fraud"
engine = get_engine()
dataframe = pd.read_sql_query("select * from {table} limit 100000".format(table=table),engine)
dataset = dataframe.values
print("First one row of the dataset")
print("Shape [{}]".format(dataset.shape))
print(dataset[0:2,:])
# split into input (X) and output (Y) variables
data_dimensions = 45
#first dimension is the index, must be removed!!!!
X = dataset[:, 1:data_dimensions]
Y = dataset[:, data_dimensions]

print("Fraud {}% ".format(float(np.sum(Y==1))*100.0/Y.shape[0]))
print("Total #samples:",Y.shape[0])
Y = to_categorical(Y, nb_classes=None)


input_dimensions = X.shape[1]
print("shapes: X[{}]=====Y[{}]".format(X.shape, Y.shape))

First one row of the dataset
Shape [(100000, 47)]
[[  4.04144290e+07   9.64125000e+05   1.37605392e+18   4.11620000e+04
    1.20000000e+01   7.20776000e+05   1.61830000e+04   4.70615000e+05
    3.67000000e+02   3.88896000e+05   2.91780000e+04   6.55000000e+02
    1.00000000e+00   8.00000000e+00   0.00000000e+00   6.00000000e+00
    0.00000000e+00   0.00000000e+00   5.00000000e+00   4.00000000e+00
    1.00000000e+00   1.00000000e+02   0.00000000e+00   0.00000000e+00
    0.00000000e+00   9.80000000e+01   1.00000000e+00   1.00000000e+00
    0.00000000e+00   3.00000000e+00   3.00000000e+00   0.00000000e+00
    1.90000000e+01   5.84100000e+03   8.26000000e+02   6.10000000e+07
    1.20000000e+01   9.51497000e+03   1.00000000e+04   1.00000000e+04
    5.85030000e+02  -9.22337204e+18   1.37341440e+18   9.60560000e+02
    1.37013120e+18   0.00000000e+00  -9.22337204e+18]
 [  4.04144770e+07   9.64125000e+05   1.38825685e+18   1.45520000e+04
    4.40000000e+01   2.03993000e+06   6.12110000e+04   1

In [69]:
def reshape_time(raw_data,seq_len=100):
    total = raw_data.shape[0]
    feat_num = raw_data.shape[1]
    seq_num = total/seq_len
    reshaped_data = raw_data.reshape((seq_num,seq_len,feat_num))
    return reshaped_data

In [68]:
def load_sql(index,table,lim = 1e5):
    dataframe = pd.read_sql_query("select * from {table} limit {lim} offset {offset}".format(table=table,lim=lim,offset=index),engine)
    dataset = dataframe.values
    # split into input (X) and output (Y) variables
    data_dimensions = 45
    #first dimension is the index, must be removed!!!!
    X = dataset[:, 1:data_dimensions]
    Y = dataset[:, data_dimensions]
    Y = to_categorical(Y, nb_classes=None)
    return X, Y

In [70]:
def load_sql_time(index,table,batch_size = 1e5):
    dataframe = pd.read_sql_query("select * from {table} order by authzn_rqst_proc_tm limit {lim} offset {offset}".format(table=table,lim=batch_size,offset=index),engine)
#     dataset = dataframe.sort_values(['authzn_rqst_proc_tm'], ascending=[1]).values
    dataset = dataframe.values
#     print (dataframe.columns)
    # split into input (X) and output (Y) variables
    data_dimensions = 45
    #first dimension is the index, must be removed!!!!
    X = dataset[:, 1:data_dimensions]
    Y = dataset[:, data_dimensions]
    Y = to_categorical(Y, nb_classes=None)
    X = reshape_time(X)
    Y = reshape_time(Y)
    return X, Y

In [46]:
#this will give chunk of 10K pictures, 100 such chunks form entire dataset:
def myGenerator(total_samples,table="data_fraud_little",batch_size=):
    engine = get_engine()
    total_samples = pd.read_sql_query("select count(*) from {table}".format(table=table),engine).values[0][0]
    print("total samples {}".format(total_samples))

    while 1:
            # following loads data from sql file numbered with fileIndex
            for fileIndex in range(0,total_samples,batch_size)
            (X_train, y_train) = load_sql_time(fileIndex,table,batch_size)
            yield X_train, y_train



In [71]:
gen = myGenerator("data_fraud_little")
for i in range(2):
    data = next(gen)
    print(data[0][0][:][:,1])

total samples 600000
[  1.36736678e+18   1.36736694e+18   1.36736704e+18   1.36736720e+18
   1.36736739e+18   1.36736793e+18   1.36736809e+18   1.36736810e+18
   1.36736811e+18   1.36736825e+18   1.36736829e+18   1.36736829e+18
   1.36736887e+18   1.36736894e+18   1.36736918e+18   1.36736990e+18
   1.36737033e+18   1.36737046e+18   1.36737053e+18   1.36737061e+18
   1.36737085e+18   1.36737127e+18   1.36737135e+18   1.36737137e+18
   1.36737140e+18   1.36737159e+18   1.36737166e+18   1.36737175e+18
   1.36737210e+18   1.36737211e+18   1.36737214e+18   1.36737234e+18
   1.36737244e+18   1.36737327e+18   1.36737344e+18   1.36737381e+18
   1.36737394e+18   1.36737394e+18   1.36737411e+18   1.36737455e+18
   1.36737483e+18   1.36737484e+18   1.36737487e+18   1.36737489e+18
   1.36737494e+18   1.36737498e+18   1.36737499e+18   1.36737511e+18
   1.36737518e+18   1.36737539e+18   1.36737540e+18   1.36737542e+18
   1.36737545e+18   1.36737545e+18   1.36737568e+18   1.36737593e+18
   1.36737692

data[0][0][0:10] returns a matrix (10,44), which can than be indexed

In [59]:
X = data[0]
data[0][0][0:10][:,1]

array([  1.36736694e+18,   1.36736704e+18,   1.36736720e+18,
         1.36736739e+18,   1.36736793e+18,   1.36736809e+18,
         1.36736810e+18,   1.36736811e+18,   1.36736825e+18,
         1.36736829e+18])

In [51]:
data[0][0][0].shape

(44,)

In [63]:
data[0].shape

(1000, 100, 44)

In [62]:
data[1].shape

(1000, 100, 2)

In [64]:
from keras.layers import TimeDistributed
from keras.layers.recurrent import SimpleRNN

In [65]:
def keras_rnn_model(input_dimensions,seq_len,hidden=100,layers=1,unit=SimpleRNN):
    # create model
    model = Sequential()
    model.add(unit(hidden, input_shape=(seq_len,input_dimensions), init='normal', activation='relu',return_sequences=True))
    if hidden is not None:
        for l in range(layers):
            model.add(unit(hidden, init='normal', activation='relu',return_sequences=True))
    model.add(TimeDistributed(Dense(2, init='normal', activation='softmax')))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='rmsprop')
    return model